## An example how to calculate sdg 11.2 indicator for Friedrichshain (Berlin)

This is an example on how to calculate the SDG11.2.1 indicator for the district of Friedrichshain in Berlin (Germany).
We start by importing all necessary libraries and setting our project path to the system paths.

In [ ]:
import os
import sys
project_path = os.path.abspath('../')
sys.path.append(project_path)
import geopandas as gpd
from matplotlib import pyplot as plt
from ptac import accessibility as accessibility
from ptac import osm as osm
from ptac import population as population

Next, we have to load public transport stops and population data.

In [7]:
stops = gpd.read_file(project_path + "/data/friedrichshain_pt.gpkg")
population_points = gpd.read_file(project_path + "/data/friedrichshain_population.gpkg")

You might want to generate a boundary of the population dataset using the convex hull function of geopandas.
The boundary is necessary as a bounding box to download the road network.

In [ ]:
boundary_gdf = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[population_points.unary_union.convex_hull])

boundary_gdf.plot()

if you have population data in a raster format you might want to convert this raster to a point geodataframe from
which you can start the accessibility calculation. You will be able to download raster population information from the
world settlement footprint in the future. We hope this functionality will be available until autumn 2021.

In [ ]:
population_points = population.raster_to_points(path=project_path + "/data/friedrichshain_population_raster.tif")

Now, we plot population and pt stops

In [ ]:
ax = population_points.plot(column= "pop", cmap='OrRd', legend=True, alpha=0.1)
ax.set_facecolor("black")
stops.plot(color="royalblue", ax=ax, markersize=6, alpha=0.6)

Calculate sum of population in the focus area

In [ ]:
population_sum = population_points["pop"].sum()

print(f"Overall sum of population is: {population_sum}")

Now calculate the accessibility from population point data to stations. If you do not pass a network_gdf dataset
, it will be downloaded automatically based on the bounding box.

In [ ]:
accessibility_output = accessibility.distance_to_closest(start_geometries=population_points,
                                  destination_geometries=stops,
                                  boundary_geometries=boundary_gdf,
                                  network_gdf=None,
                                  verbose=1)

the output looks like this:

In [ ]:
print(accessibility_output.head(5))

It is also possible to download the network before calculating

In [ ]:
network_gdf = osm.get_network(boundary_gdf)

Then, we can have a look at the network..

In [ ]:
ax = stops.plot(color="black", markersize=6, alpha=0.6)
network_gdf.plot(color="grey", linewidth=0.2, ax=ax)
plt.show()

.. and store it to disk. You are able to drag and drop this data set into your GIS software.

In [ ]:
network_gdf.to_file(project_path + "/data/friedrichshain_network.gpkg", driver="GPKG")

Now, we can calculate again with our stored network.

In [ ]:
network_gdf = gpd.read_file(project_path + "/data/friedrichshain_network.gpkg")

accessibility_output= accessibility.distance_to_closest(start_geometries=population_points,
                                  destination_geometries=stops,
                                  boundary_geometries=boundary_gdf,
                                  maximum_distance=500,
                                  network_gdf=network_gdf,
                                  verbose=1)

Setting maximum distance to 500 meters generates an output in
which only includes starting points which are within 500 meters from the next station.

In [ ]:
print(accessibility_output.head(10))

Calculate sum of population with access to pt within 500 meters

In [ ]:
sum_pop_with_access = accessibility_output["pop"].sum()
print(sum_pop_with_access)

Calculate the actual indicator and print the result

In [ ]:
sdg = accessibility.calculate_sdg(df_pop_total=population_points,
                                  pop_accessible=accessibility_output,
                                  population_column='pop')
print(f"{sdg} percent of the population have access to the public transit system within 500 meters")


Plot result as a map

In [ ]:
ax = population_points.to_crs(4326).plot(color="grey")
accessibility_output.to_crs(4326).plot(column='distance_pt', cmap='OrRd', ax=ax, legend=True)
plt.show()


If you wish to differentiate between high and low capacity public transport you should split the
dataset in. This is an example how to do it. Might be different in your case based on the structure of your data.

In [ ]:
stops_low_capacity = stops[stops["highway"]=="bus_stop"]
stops_high_capacity = stops[(stops["railway"]=="tram_stop")|(stops["railway"]=="station")]


Now, we have to run accessibility calculation for high and low capacity public transport

In [ ]:
network_gdf = gpd.read_file(project_path + "/data/friedrichshain_network.gpkg")

accessibility_output_low = accessibility.distance_to_closest(start_geometries=population_points,
                                  destination_geometries=stops_low_capacity,
                                  transport_system="low-capacity",
                                  boundary_geometries=boundary_gdf,
                                  network_gdf=network_gdf,
                                  verbose=1)


accessibility_output_high= accessibility.distance_to_closest(start_geometries=population_points,
                                  destination_geometries=stops_high_capacity,
                                  transport_system="high-capacity",
                                  boundary_geometries=boundary_gdf,
                                  network_gdf=network_gdf,
                                  verbose=1)

.. and calculate the actual SDG indicator out of that. As assumed, the value is a bit higher, now.

In [ ]:
sdg_low_high = accessibility.calculate_sdg(df_pop_total=population_points,
                                           pop_accessible=[accessibility_output_high, accessibility_output_low],
                                           population_column='pop')
print(f"{sdg_low_high} percent of the population have access to low- and high-capacity public transit systems")
